In [ ]:
# PLOT Plank Blackbody curve including subinterval version 2.0

#%matplotlib inline
%matplotlib widget
#%matplotlib notebook

from ipywidgets import *
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt

#MAIN SECTION ****************************************************************

#base directory
bdir="C:\\Users\Owner\Documents\JWD\Calculators\Tools\BAFDR"

# Load Modtran data
modinputdata = bdir + r"\ModtranDataTransposed.csv"
modinputdesc = bdir + r"\Modtrandesc.csv"
#print(modinputdata)

MTD = np.loadtxt(modinputdata, delimiter=',')
with open(modinputdesc, 'r') as file:
    MTdesc = file.readlines()

# Load BBATM inputs
desc_file_path = bdir + r'\BAFDR_S_inputs.txt'

with open(desc_file_path, 'r') as file:
    # Read lines and split into descriptor and value using colon as separator
    lines = [line.strip().split(':') for line in file.readlines()]

# Extract values (assuming each line has only one value)
values = [line[1].strip() for line in lines]

# Convert the list of strings to a NumPy array
inputs_array = np.array(values)

# DEBUG ONLY Print the shape and values of the array
#print("Shape of the array:", inputs_array.shape)
#print("Values of the array:")
#print(inputs_array)

# Extract input values
plot_units, mtnum, emit, Temp, wvl1, wvl2, f, D, p, tran, DWL1, DWL2, ScaleFactor, QE, OperT,Vn_Amp_Ampin,Vmax,Res_ADC,CifF,Tint = map(float, inputs_array[:20])
mtnum = int(mtnum)
Res_ADC = int(Res_ADC)



In [ ]:
# BLackbody and Atmosphere Section ***************************************************************
# Check for valid inputs
if Temp < 4.0 or emit <= 0 or wvl1 <= 0 or wvl2 <= 0:
    print('Invalid inputs')
else:
    # Wavelength lower limit, upper limit, and step size
    # These must match the modtran file
    lowl, upl, step = 1.0, 15.025, 0.025
    pi = np.pi

    # Step out wavelengths, match array size of L to wvl
    wvl = np.arange(lowl, upl, step)
    # Set arrays equal in size to wvl
    L, LW = wvl * 3, wvl * 4
    MLsubband, MLsubbandW = 0.0, 0.0

    # Compute radiant intensity values L and sub-band total
    for count, i in enumerate(wvl):
        LW[count] = 37400 / (wvl[count]**5) * (1 / ((2.71828)**(14400 / (wvl[count] * Temp)) - 1))
        L[count] = (1.88e27 / (wvl[count]**4)) * (1 / (np.exp(14388 / (wvl[count] * Temp)) - 1)) / 10000
        if wvl1 < wvl[count] < wvl2:
            if mtnum == -1:
                MLsubband += L[count] * step
                MLsubbandW += LW[count] * step
            else:
                MLsubband += L[count] * step * MTD[mtnum, count]
                MLsubbandW += LW[count] * step * MTD[mtnum, count]

    # Sum of radiances
    fullrad, fullradW = sum(L) * step, sum(LW) * step

    # Radiance times atmospheric transmission
    if mtnum == -1:
        ML, MLW = L, LW
        MTdesc[-1] = "Perfect Transmission"
    else:
        ML, MLW = L * MTD[mtnum], LW * MTD[mtnum]

    # Factor in emittance
    MLe, MLWe = ML * emit, MLW * emit

    # Do for sub band
    MLsubbande, MLsubbandWe = emit * MLsubband, emit * MLsubbandW

    # Convert to radiance
    MLsubbandes, MLsubbandWes = emit * MLsubband / pi, emit * MLsubbandW / pi

    # Find peak L and index of peak
    mrx, mrxw = max(L), max(LW)
    inpeak, inpeakw = np.argmax(L), np.argmax(LW)

    # Make a vertical line
    vlinex = np.repeat(wvl[inpeak], 5)
    vliney = vlinex * 3
    for count, i in enumerate(vlinex):
        vliney[count] = (count / 3) * 2 * L[inpeak]

    vlinex1, vlinex2 = np.repeat(wvl1, 5), np.repeat(wvl2, 5)

    # Format output
    fullrad_d, Lsubband_d = "{:.2E}".format(fullrad), "{:.2E}".format(MLsubband)
    fullradW_d, LsubbandW_d = "{:.2E}".format(fullradW), "{:.2E}".format(MLsubbandW)
    mrxd, wvl1d, wvl2d = round(mrx, 2), round(wvl1, 2), round(wvl2, 2)
    wvlpd, wvlpdw = round(wvl[inpeak], 2), round(wvl[inpeakw], 2)
    peakph, peakW = 3666 / Temp, 2898 / Temp

    # Output totals
    CalcResults = widgets.HTML(value="<p style='font-size:16pt'><b>BB x ATM Calculated Results:</b>")
    #display(CalcResults)  # Displays calculated results message
    #print("Atmosphere = ",MTdesc[int(mtnum)])
    print(" Exitance= %5.4e Watts/cm^2       Sub Band: %4.1f - %4.1f um Exitance = M = %5.4e Watt/cm^2   "
          % (fullradW, wvl1, wvl2, MLsubbandW))
    print(" Peak Wvl= %5.2f (Watts)               Sub Band: %4.1f - %4.1f um Radiance = L = %5.4e Watt/cm^2-str"
          % (peakW, wvl1, wvl2, MLsubbandWes))
    print(" Exitance= %5.4e ph/sec-cm^2      Sub Band: %4.1f - %4.1f um Exitance = M = %5.4e ph/sec-cm^2  "
          % (fullrad, wvl1, wvl2, MLsubband))
    print(" Peak Wvl= %5.2f (Photons)             Sub Band: %4.1f - %4.1f um Radiance = L = %5.4e ph/sec-cm^2-str"
          % (peakph, wvl1, wvl2, MLsubbandes))

    

In [ ]:
# Blackbody and Atmosphere plotting Section **************************************************************
if plot_units > 0 : 
        #plotting section
        plt.style.use('dark_background')
        fig = plt.figure(figsize=(13, 7))
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(vlinex1, vliney, '--', color='white', linewidth=1.5)
        ax.plot(vlinex2, vliney, '--', color='white', linewidth=1.5)
        plt.xlabel('Wavelength ' + chr(955) + ' um')

        # Major ticks every 20, minor ticks every 5
        major_xticks = np.arange(1, 15, 1)
        minor_xticks = np.arange(1, 15, 0.25)
        ax.set_xticks(major_xticks)
        ax.set_xticks(minor_xticks, minor=True)
        ax.grid(which='minor', alpha=0.25)
        ax.grid(which='major', alpha=0.75)

    
        title = "Temp=" + str(Temp) + "K     Atmosphere: " + MTdesc[mtnum]
        plt.title(title)

        if plot_units == 1:  # "Photon/Sec-cm^2 semilog":
            plt.yscale('log')
            plt.axis([1.0, 15.0, 10E-8 * max(L), 2 * max(L)])
            plt.ylabel('ph/sec-cm^2-um', color='red')
            plt.plot(wvl, MLe, 'red', linewidth=1.25, alpha=0.75)
            plt.plot(wvl, L, 'g:')
            plt.plot(wvl[inpeak], L[inpeak], 'gv')
            plt.text(1.02 * wvl[inpeak], L[inpeak], "       " + str(round(peakph, 2)) + "um", color='green')
            plt.fill_between(wvl, 0, MLe, where=(wvl >= wvl1), facecolor='blue', alpha=0.6)
            plt.fill_between(wvl, 0, MLe, where=(wvl >= wvl2), facecolor='black')

        elif plot_units == 2:  # "Photon/Sec-cm^2":
            plt.axis([lowl, upl, 0, 1.1 * max(L)])
            plt.ylabel('ph/sec-cm^2-um', color='red')
            plt.plot(wvl, MLe, 'red', linewidth=1.25, alpha=0.75)
            plt.plot(wvl, L, 'g:')
            plt.plot(wvl[inpeak], L[inpeak], 'gv')
            plt.text(1.02 * wvl[inpeak], L[inpeak], "       " + str(round(peakph, 2)) + "um", color='green')
            plt.fill_between(wvl, 0, MLe, where=(wvl >= wvl1), facecolor='blue', alpha=0.6)
            plt.fill_between(wvl, 0, MLe, where=(wvl >= wvl2), facecolor='black')

        else:
            print('invalid plot type')

        # Save Figure
        bboutplt = bdir + r'\BBATMplt.png'
        #print(bboutplt)
        plt.savefig(bboutplt)
    
        # Save Radiance  to a CSV file
        # First stack wvl and MLe horizontally
        data_to_save = np.column_stack((wvl, MLe))
        # Save wvl and MLe to a CSV file
        output_file_path = bdir + r'\BBATMplt_data.csv'
        np.savetxt(output_file_path, data_to_save, delimiter=',')
    
  

In [ ]:
# FLUX Section ******************************************************************
# Compute Flux on detector
Flux = (tran * MLsubbandes * pi * (p * 1e-04)**2 * (D * .01)**2) / (4 * (f * 0.01)**2)  

print("Flux on detector = ", "{:.4e}".format(Flux), " photons/sec")

In [ ]:
# Detector Section ******************************************************************
# Compute Signal and Dark Currents
# Constants
q = 1.602e-19 # [C], Elementary charge
kB = 1.38e-23 # [J/K], Boltzmann's constant

# Empirical parameters from Rule 07-Revisited: Journal of ELECTRONIC MATERIALS, Vol. 39, No. 7, 2010

J0 = 8367.00001853855 # [A/cm2]
Pwr = 0.544071281108481
C = -1.16239134096245
WLscale = 0.200847413564122 #[um] 
WLthreshold = 4.63513642316149 # [um]
    
#Computations
Isig=Flux*QE*q
Jsig=Isig/(p*1e-4)**2
    
if DWL2 > WLthreshold :
        WLe = DWL2
else : 
        tcalc = WLscale/DWL2-WLscale/WLthreshold
        tcalc2 = np.exp(Pwr*np.log(tcalc))
        WLe=DWL2/(1-tcalc2)
        print ('Equiv Wvl Invoked wvl = ',WLe)
    
#print(ScaleFactor,J0,C,q,kB,WLe,OperT)
Jdark=ScaleFactor*J0*np.exp(C*1.24*q/(kB*WLe*OperT))
Idark=Jdark*(p*1e-4)**2
    
TBLIP=C*1.24*q/(kB*DWL2*np.log(Jsig/(J0*ScaleFactor)))
MaxBLIP=C*1.24*q/(kB*DWL2*np.log(Jsig/(J0*ScaleFactor)/(QE)))
#print ("  Isig=",Isig,"  Jsig=",Jsig," Idark=", Idark, "  JDark=", Jdark)
#print ("TBLIP = ", TBLIP, "MaxBLIP=", MaxBLIP)
     

In [ ]:
#ROIC Section **************************************************************************
#BEGIN ROIC CALCULATIONS
   
# Calculation vector parameters for graphing
t = np.logspace(-9,-1,1000) # [s], Integration time vector
t1 = np.ones(t.shape) # Ones vector of size(t)
Rdata= 5000
Npix = 1
    
#Ci = CifF * 1e-15 # [F], Integration capacitance (get value in F  why not 1e-16?)
Ci = CifF * 1e-16 # [F], Integration capacitance (get value in F)
Vn_Amp_Ampin = Vn_Amp_Ampin * t1 # [Vrms], Amplifier noise referred to amplifier input (multiply by ones vector)
    
# Total pixel current
Itot = Isig + Idark # [A]

# ADC conversion gain
G_ADC = 2**Res_ADC / Vmax # [DU/V], ADC gain

# Quantization noise
#Dn_quant_out = 1/np.sqrt(12) * t1 # [DUrms]  why is this 12???
Dn_quant_out = 1/np.sqrt(Res_ADC) * t1 # [DUrms]

# Integrated pixel voltage
Vtot_pix = np.minimum(Itot * t /Ci,Vmax) # [V], Total
Vsig_pix = Vtot_pix * Isig/Itot # [V], Signal only

# Pixel noise sources
Vn_shot_pix = np.sqrt(q * Vtot_pix /Ci) * t1 # [Vrms], Shot noise
Vn_kTC_pix = np.sqrt(kB * OperT /Ci) * t1 # [Vrms], kTC (reset) noise

# Propagate signal to output
Dsig_out = Vsig_pix * G_ADC # [DU]

# Propagate all noise sources to output
Dn_shot_out = Vn_shot_pix * G_ADC # [DUrms]
Dn_kTC_out = Vn_kTC_pix * G_ADC # [DUrms]
Dn_Amp_out = Vn_Amp_Ampin * G_ADC # [DUrms]

# Sum all noises at output
Dntot_out = np.sqrt(Dn_shot_out**2 + Dn_kTC_out**2 + Dn_Amp_out**2 + Dn_quant_out**2) # [DUrms]

# Compute SNR
SNR = 20*np.log10(Dsig_out/Dntot_out) # [dB]


## Compute other results

# Well capacity and well fill
Nwell = Ci * Vmax / q # [#e-]
wf = str(round(100*(Tint * Itot)/(q*Nwell),2))

# Read noise in electrons
Nnread = Dntot_out[0] / G_ADC * Ci / q # [#e-]

# Max SNR
SNRmax = np.amax(SNR) # [dB]

# Dynamic range
DR = 20*np.log10(np.amax(Dsig_out)/Dntot_out[0]) # [dB]

# Saturation time
tSat = Nwell * q / Itot # [s]   

# Read time
#tRead = Npix * Res_ADC / Rdata # [s]

# Max frame rate (no integration)
#FRmax0 = 1/tRead # [fps]

# Max frame rate (full well, IWR)
#FRmax_MW_IWR = 1/np.maximum(tSat,tRead) # [fps]

# Max frame rate (full well, ITR)
#FRmax_MW_ITR = 1/(tSat+tRead) # [fps]

        
i=0
index = 77777  #?  must be great than size Tint ie 1000?
for i in range (0, 1000):
        if t[i] < Tint:
            i+=1
        else:
            index=i
            break

## Define limiting noise string

# Determine limiting noise
DnmaxIndex = np.argmax((Dn_kTC_out[0],Dn_Amp_out[0],Dn_quant_out[0],Dn_shot_out[i])) 

# Define string
if DnmaxIndex == 0:
        LimitingNoise = 'reset'
elif DnmaxIndex == 1:
        LimitingNoise = 'amplifier'
elif DnmaxIndex == 2:
        LimitingNoise = 'quantization' 
elif DnmaxIndex == 3:
        LimitingNoise = 'SHOT'

            
## "ROIC Calculated Results" section heading and text results
CalcResults = widgets.HTML(value = "<br><p style='font-size:16pt'><b>Calculated Results:</b></p>"
        "<b>Well capacity</b>: " + str(round(Nwell/1e6,2)) + " Me-  <b> Well fill: </b>" + wf + " %   <b>      Read noise</b>: " + str(int(Nnread)) + " e-" + "&emsp;  Limited by <b><i>" + LimitingNoise + "</i></b> noise<br>"
        "<b>Max SNR = </b> " + str(round(SNRmax,1)) +   "dB <b> Dynamic range = </b> " + str(round(DR,1)) + " dB <b>  SNR = </b> " + str(round(SNR[index],2)) + 'dB <b> at Tint = </b>' + str(round(Tint/1e-06,2)) + 'usec <r>:'
        #"<b>Max frame rate (no integration)</b>: " + str(round(FRmax0,1)) + " fps<br>"
        #"<b>Max frame rate (full well, IWR)</b>: " + str(round(FRmax_MW_IWR,1)) + " fps<br>"
        #"<b>Max frame rate (full well, ITR)</b>: " + str(round(FRmax_MW_ITR,1)) + " fps<br>"
        )
      
display(CalcResults)    



In [ ]:
#ROIC Plotting Section *************************************************************************
## Plot all noise sources (with sum) and signal, at output and referred to input
if plot_units > 0 :  
        plt.style.use('classic')
        fig = plt.figure(figsize=(16,9));
        ax1 = fig.add_subplot(1,2,2)
        plt.xlim((1e-07,0.1))
        ax1.loglog(t,Dn_shot_out,'-r',label='Shot noise',linewidth=3)
        ax1.loglog(t,Dn_kTC_out,'-m',label='Reset noise',linewidth=1.25)
        ax1.loglog(t,Dn_Amp_out,'-c',label='Amplifier noise',linewidth=1.25)
        ax1.loglog(t,Dn_quant_out,'-b',label='Quantization noise',linewidth=1.25)
        ax1.loglog(t,Dntot_out,'-k',label='TOTAL noise',linewidth=4)
        ax1.loglog(t,Dsig_out,'-g',label='Signal',linewidth=4)
        duliney = [0.0001, 1, 100000]
        dulinex = [Tint, Tint, Tint]
        ax1.loglog(dulinex, duliney,'--k')
        if Tint < 1e-3:
            plt.text(Tint*1.5, 2e-4, "Tint = " + str(round(Tint/1e-3,2)) + 'msec',color='grey',fontsize=15)
        else:
            plt.text(Tint*1e-2, 2e-4, "Tint = " + str(round(Tint/1e-3,2)) + 'msec',color='grey',fontsize=15)
        ax1.grid()
        ax1.set_xlabel('Integration time [s]')
        ax1.set_ylabel('Output noise [DU]')
        ax1.legend(loc='upper left',fontsize='medium')
        #plt.title('Noise budget')

        # Integration time [s] to signal [#e-] conversion function for secondary x-axis
        def t2sig(x):
            return x * Isig / q

        # Signal [#e-] to integration time [s] conversion function for secondary x-axis
        def sig2t(x):
            return x * q / Isig

        # Output [DU] to input [#e-] conversion function for secondary y-axis
        def out2in(x):
            return x / G_ADC * Ci / q

        # Input [#e-] to output [DU] conversion function for secondary y-axis
        def in2out(x):
            return x * q / Ci * G_ADC

        # Secondary axes
        secaxx = ax1.secondary_xaxis('top', functions=(t2sig, sig2t)) # x-axis in units of #e-
        secaxy = ax1.secondary_yaxis('right', functions=(out2in, in2out)) # y-axis in units of #e-
        secaxx.set_xlabel('Signal [#e-]')
        secaxy.set_ylabel('Input-referred noise [#e-]')

        #plt.figure(2);
        plt.style.use('dark_background')
        ax2=fig.add_subplot(1,2,1)
        ax2.semilogx(t,SNR,color='orange',linewidth=4)
        vlinex = np.repeat(Tint,3)
        vliney = [0, 50, 100] 
        hlinex = [1e-9, 1e-5, 1e-1]
        hliney = np.repeat(SNR[index],3)
        if SNR[index] >=0:
            plt.plot (vlinex,vliney,'--',color='grey',linewidth=2)
            plt.plot (hlinex,hliney,'--',color='grey',linewidth=2)
            plt.text(1.25e-08, SNR[index]+1, "SNR = " + str(round(SNR[index],2)) + 'db',color='orange',fontsize=15)
        if Tint < 1e-3:
            plt.text(Tint*1.6, 2, "Tint = " + str(round(Tint/1e-3,2)) + 'msec',color='orange',fontsize=15)
        else:
            plt.text(Tint*1e-2, 2, "Tint = " + str(round(Tint/1e-3,2)) + 'msec',color='orange',fontsize=15)

        plt.ylim((0,DR))
        plt.xlim((1e-07,0.1))
        ax2.grid()
        ax2.set_xlabel('Integration time [s]')
        ax2.set_ylabel('SNR [dB]')
        plt.title('Signal-to-noise ratio')

        # Save Figure
        ROICoutplt = bdir + r'\ROICplt.png'
        plt.savefig(ROICoutplt)
    
   
   

In [ ]:
#Radiometery Calculations section ****************************************************
    
#constants
c=2.9979e8 #m/sec
c1=3.7418e4 #watt-um^4/cm^2
c2=1.4388e4 #um-K
h=6.6e-34 # m2 kg / s  plancks constant
q=1.6e-19 #coulomg charge of e
    
#planck derivative 
wvl = ((wvl2-wvl1)/2 + wvl1)
wvlm = wvl*1e-6
t1 = np.exp(c2/(wvl*Temp))  #unitless - temporary constant for eqn below
#print((wvl*Temp),c2,(c2/(wvl*Temp)),'t1=',t1)
t2= c1/wvl**5
t3= (1/(t1 - 1))
M = t2*t3 # Watt/(cm^2 - um) M 5.11  pg126
L=M/np.pi
PLDR2 = (L*c2*t1)/(wvl*(Temp**2)*(t1 - 1)) #planck function deriviative Holst yello textbook 3.46 pg55 
#print('Temp=',Temp,'wvl=',wvl,'L= ',L, '  PLDR= ', PLDR2)
    
#Caclulate number of Photons, Photoelectroncs, Noise electrons and SNR
NPe = Isig*Tint/q #Photo electrons
NPhoton = NPe/QE #back out incident photons
PNe = np.sqrt((NPhoton))  # Photon Shot Noise
DNe = np.sqrt((Idark*Tint)/q)  # Dark Current Noise
TNe = np.sqrt((DNe**2 + PNe**2 + Nnread**2)) #add noise in quadrature - Dark Noise e + photon/flux noise e + ROIC fixed noise floor
NEC = TNe # Noise Equiv Carriers = Total Noise Electrons 
NEI = NEC/(QE*Tint*(p*1e-4)**2)
SNR2 = NPhoton/NEC  #Nphoto-electrons over total noise electrons
SNR2L = 20*np.log10(SNR2)
#print(Isig,Idark,SNR,NEC)
    
#DSTAR calc
NEP = NEI*(h*c/wvlm)*((p*1e-4)**2)  #Noise Equiv Power = NEI(hc/wvlm)(Area-det) 
NEB = 1/(2*Tint) #Noise Equiv Bandwidth
DSTAR = ((p*1e-4)* np.sqrt(NEB))/NEP  # cm-Hz^1/2 per Watt  D* eqn 8.24 2nd edition
    
#NETD calc
num =((f/D)**2)*np.sqrt(1/(2*Tint))   #numerator
den = tran*p*1e-4*DSTAR*PLDR2  #denominator
NETD = (4*num)/(np.pi*den)  #NETD formula 14.17 in 2nd edition
    
#show results
print("SNR=", "{:.0f}".format(SNR2),"SNR_log = ",round(SNR2L,2),"db  NEC = ","{:.0f}".format(NEC),"e  NEI=","{:.2e}".format(NEI),"Photon/Sec-cm^2")
print("D*=", "{:.2e}".format(DSTAR),"cm-Hz^1/2 per Watt (Jones)   NEP=", "{:.2e}".format(NEP),"Watts")
    
print("Noise Equivelant Temperature Difference (NETD) = ", "{:.2f}".format(NETD*1000), "mK approx for band center")
    
    

In [ ]:
#OUTPUT Section ****************************************************************************
# save RESULTS TO FILE
# Output file path
output_file_path = bdir + r'\BAFDR_S_output.txt'

# Open the file in 'w' mode to write the values
with open(output_file_path, 'w') as output_file:
        # Writing the values to the file
        output_file.write(f"Blackbody and Atmosphere:\n")
        output_file.write(f"Radiance={MLsubbandes:.4e}  Flux={Flux:.4e}\n")
       
        output_file.write(f"Detector Section:\n")
        output_file.write(f"Isig={Isig:.4e}  Jsig={Jsig:.4e}\n")
        output_file.write(f"Idark={Idark:.4e}  Jdark={Jdark:.4e}\n")
        output_file.write(f"TBLIP={TBLIP:.4e}  MaxBLIP={MaxBLIP:.4e}\n")
        
        output_file.write(f"ROIC Section:\n")
        output_file.write(f"Well Cap e- = {Nwell:.2e}  Well Fill % = {wf}\n")
        output_file.write(f"ReadNoise e- = {Nnread:.2f}  Limiting Noise = {LimitingNoise}\n")
        output_file.write(f"MaxSNR = {round(SNRmax,2)}  db Dynamic Range = {round(DR,2)}  SNR = {round(SNR[index],2)}\n")
    
        output_file.write(f"Radiometric Section:\n")
        output_file.write(f"NEC={NEC:.2e}  NEI={NEI:.4e}\n")
        output_file.write(f"DSTAR={DSTAR:.2e}  NEP={NEP:.4e}\n")
        output_file.write(f"NETD={NETD:.2e}\n")
    